In [1]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from pathlib import Path
import torch.nn.functional as F
import os
import gc
from tqdm import tqdm
import time
import random
import statistics # For mean and stdev

In [2]:

# --- Configuration ---
FEATURE_ROOT = Path("/data/kuang/Projects/MedSAM/data/BrainAGE_preprocessed")
CSV_PATH = Path("/data/kuang/Projects/MedSAM/data/Subject_demographics_info_brain_age.csv")
RESULTS_DIR = Path("./mlp_comparison_results_5runs") # Directory for results
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SPLITS = ["train", "validation", "test"]
NUM_RUNS = 5 # Number of times to train each model

# --- Hyperparameters ---
EPOCHS = 500 # Adjust as needed
LEARNING_RATE = 1e-4
BATCH_SIZE = 32
WEIGHT_DECAY = 1e-5
DROPOUT_RATE = 0.3
SCHEDULER_PATIENCE_PERCENT = 0.10
EARLY_STOPPING_PATIENCE = 40
NUM_WORKERS = 4
ATTENTION_HEADS = 8

# Create results directory
RESULTS_DIR.mkdir(exist_ok=True)

print(f"Using device: {DEVICE}")
print(f"Feature root: {FEATURE_ROOT}")
print(f"CSV path: {CSV_PATH}")
print(f"Results will be saved in: {RESULTS_DIR}")
print(f"Number of runs per model: {NUM_RUNS}")

# --- Function to set random seeds ---
def set_seed(seed):
    """Sets random seed for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
    # Ensure deterministic behavior for cuDNN
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False # Can impact performance, but needed for full determinism
    print(f"Set random seed to {seed}")



Using device: cuda
Feature root: /data/kuang/Projects/MedSAM/data/BrainAGE_preprocessed
CSV path: /data/kuang/Projects/MedSAM/data/Subject_demographics_info_brain_age.csv
Results will be saved in: mlp_comparison_results_5runs
Number of runs per model: 5


In [3]:

# --- Dataset Definition (Identical to previous) ---
class AgePredictionDataset(Dataset):
    def __init__(self, npy_dir, csv_path, split_name):
        self.npy_dir = Path(npy_dir)
        csv_path = Path(csv_path)
        self.split_name = split_name

        # print(f"\n[Dataset: {self.split_name}] Loading from NPY dir: {self.npy_dir}") # Less verbose during multi-runs
        # print(f"[Dataset: {self.split_name}] Loading metadata from CSV: {csv_path}")

        if not self.npy_dir.is_dir():
            raise FileNotFoundError(f"NPY directory not found: {self.npy_dir}")
        if not csv_path.is_file():
            raise FileNotFoundError(f"CSV file not found: {csv_path}")

        try:
            df = pd.read_csv(csv_path)
            self.meta_dict = df.set_index('filename')['age'].to_dict()
            # print(f"[Dataset: {self.split_name}] Loaded metadata for {len(self.meta_dict)} subjects from CSV.")
        except Exception as e:
            raise ValueError(f"Error loading or processing CSV {csv_path}: {e}")

        all_npy_files = list(self.npy_dir.glob("*.npy"))
        self.valid_files = []
        missing_meta_count = 0
        for npy_file in all_npy_files:
            original_filename_key = npy_file.name[:-4]
            if original_filename_key in self.meta_dict:
                self.valid_files.append(npy_file)
            else:
                missing_meta_count += 1

        # if missing_meta_count > 0:
        #     print(f"[Dataset: {self.split_name}] Warning: {missing_meta_count} .npy files did not have corresponding metadata.")

        if not self.valid_files:
             raise RuntimeError(f"No valid .npy files found in {self.npy_dir} that match the metadata.")

        # print(f"[Dataset: {self.split_name}] Found {len(self.valid_files)} valid .npy files with metadata.")

    def __len__(self):
        return len(self.valid_files)

    def __getitem__(self, idx):
        npy_path = self.valid_files[idx]
        original_filename_key = npy_path.name[:-4]

        try:
            embedding = np.load(npy_path)
            embedding_tensor = torch.tensor(embedding, dtype=torch.float32)

            if embedding_tensor.shape == (1, 256, 64, 64):
                 pooled_embedding = F.adaptive_avg_pool2d(embedding_tensor, (1, 1)).squeeze()
            elif embedding_tensor.shape == (256, 64, 64):
                 pooled_embedding = F.adaptive_avg_pool2d(embedding_tensor.unsqueeze(0), (1, 1)).squeeze()
            elif embedding_tensor.shape == (256,):
                 pooled_embedding = embedding_tensor
            else:
                 # print(f"Warning: Unexpected embedding shape {embedding_tensor.shape} for {npy_path}. Trying to squeeze.") # Less verbose
                 pooled_embedding = embedding_tensor.squeeze()
                 if pooled_embedding.ndim == 0 or pooled_embedding.shape[0] != 256:
                      raise ValueError(f"Embedding shape after squeeze is not 256 for {npy_path}: {pooled_embedding.shape}")

            age = self.meta_dict[original_filename_key]
            age_tensor = torch.tensor(age, dtype=torch.float32)

            return pooled_embedding, age_tensor

        except Exception as e:
            print(f"Error loading or processing file {npy_path} for subject {original_filename_key}: {e}")
            raise RuntimeError(f"Failed to load/process {npy_path}") from e



In [4]:

# --- Model Definitions (Identical to previous) ---
class AgeMLP_2Layer(nn.Module):
    def __init__(self, input_dim=256, hidden_dim1=128, hidden_dim2=64, dropout_rate=DROPOUT_RATE):
        super(AgeMLP_2Layer, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, hidden_dim1), nn.ReLU(), nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim1, hidden_dim2), nn.ReLU(), nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim2, 1)
        )
    def forward(self, x): return self.mlp(x).squeeze(-1)

class AgeMLP_3Layer(nn.Module):
    def __init__(self, input_dim=256, hidden_dim1=128, hidden_dim2=64, hidden_dim3=32, dropout_rate=DROPOUT_RATE):
        super(AgeMLP_3Layer, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, hidden_dim1), nn.ReLU(), nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim1, hidden_dim2), nn.ReLU(), nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim2, hidden_dim3), nn.ReLU(), nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim3, 1)
        )
    def forward(self, x): return self.mlp(x).squeeze(-1)

class AgeMLP_3Layer_AttentionBN(nn.Module):
    def __init__(self, input_dim=256, embed_dim=256, num_heads=ATTENTION_HEADS,
                 hidden_dim1=128, hidden_dim2=64, hidden_dim3=32, dropout_rate=DROPOUT_RATE):
        super(AgeMLP_3Layer_AttentionBN, self).__init__()
        if embed_dim % num_heads != 0: raise ValueError(f"embed_dim ({embed_dim}) must be divisible by num_heads ({num_heads})")
        self.attention = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, batch_first=False)
        self.norm_attn = nn.LayerNorm(embed_dim)
        self.mlp = nn.Sequential(
            nn.Linear(input_dim, hidden_dim1), nn.BatchNorm1d(hidden_dim1), nn.ReLU(), nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim1, hidden_dim2), nn.BatchNorm1d(hidden_dim2), nn.ReLU(), nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim2, hidden_dim3), nn.BatchNorm1d(hidden_dim3), nn.ReLU(), nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim3, 1)
        )
    def forward(self, x):
        x_attn = x.unsqueeze(0); attn_output, _ = self.attention(x_attn, x_attn, x_attn)
        x = self.norm_attn(x_attn + attn_output); x = x.squeeze(0)
        output = self.mlp(x); return output.squeeze(-1)



In [5]:

# --- Training and Evaluation Functions (Identical to previous) ---
def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train(); total_loss = 0.0; num_samples = 0
    pbar = tqdm(loader, desc="Training", leave=False)
    for features, ages in pbar:
        features, ages = features.to(device), ages.to(device)
        optimizer.zero_grad(); predictions = model(features)
        loss = criterion(predictions, ages); loss.backward(); optimizer.step()
        total_loss += loss.item() * features.size(0); num_samples += features.size(0)
        pbar.set_postfix(loss=loss.item())
    return total_loss / num_samples if num_samples > 0 else 0

@torch.no_grad()
def evaluate(model, loader, criterion, device):
    model.eval(); total_loss = 0.0; total_mae = 0.0; num_samples = 0
    pbar = tqdm(loader, desc="Evaluating", leave=False)
    for features, ages in pbar:
        features, ages = features.to(device), ages.to(device)
        predictions = model(features); loss = criterion(predictions, ages)
        total_loss += loss.item() * features.size(0)
        mae = F.l1_loss(predictions, ages, reduction='sum')
        total_mae += mae.item(); num_samples += features.size(0)
        pbar.set_postfix(loss=loss.item(), mae=(total_mae/num_samples if num_samples > 0 else 0))
    avg_loss = total_loss / num_samples if num_samples > 0 else 0
    avg_mae = total_mae / num_samples if num_samples > 0 else 0
    return avg_loss, avg_mae

# --- Modified Training and Evaluation Loop for Single Run ---
def train_evaluate_single_run(model_class, model_name, run_number, loaders, device):
    """Trains and evaluates a model for a single run with a specific seed."""
    print(f"\n--- Starting Run {run_number+1}/{NUM_RUNS} for Model: {model_name} ---")
    current_seed = run_number # Use run number as seed for simplicity
    set_seed(current_seed)

    model_save_path = RESULTS_DIR / f"best_model_{model_name}_run_{run_number}.pth"

    model = model_class().to(device)
    criterion = nn.L1Loss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler_patience_epochs = max(1, int(EPOCHS * SCHEDULER_PATIENCE_PERCENT))
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=scheduler_patience_epochs, factor=0.5, verbose=False) # Less verbose scheduler

    best_val_mae = float('inf')
    epochs_no_improve = 0
    best_epoch = -1
    train_start_time = time.time()

    # print(f"Model save path: {model_save_path}") # Less verbose

    for epoch in range(EPOCHS):
        # epoch_start_time = time.time() # Less verbose
        train_loss = train_one_epoch(model, loaders['train'], criterion, optimizer, device)
        val_loss, val_mae = evaluate(model, loaders['validation'], criterion, device)
        # epoch_duration = time.time() - epoch_start_time # Less verbose

        # Less verbose epoch print
        if (epoch + 1) % 10 == 0 or epoch == 0 or epoch == EPOCHS -1 : # Print every 10 epochs + first/last
             print(f"  Epoch {epoch+1}/{EPOCHS} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val MAE: {val_mae:.3f}")

        scheduler.step(val_loss)

        if val_mae < best_val_mae:
            best_val_mae = val_mae
            best_epoch = epoch + 1
            epochs_no_improve = 0
            try:
                torch.save(model.state_dict(), model_save_path)
                # print(f"  -> New best Val MAE: {best_val_mae:.3f}. Saved model.") # Less verbose
            except Exception as e:
                 print(f"  -> Error saving model: {e}")
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= EARLY_STOPPING_PATIENCE:
            print(f"  Early stopping triggered at epoch {epoch+1}.")
            break

        # Optional: Clean up GPU memory periodically
        if (epoch + 1) % 50 == 0: # Less frequent cleanup
             gc.collect()
             if device.startswith('cuda'):
                 torch.cuda.empty_cache()

    train_duration = time.time() - train_start_time
    print(f"--- Run {run_number+1} Finished for {model_name} (Time: {train_duration:.2f}s) ---")

    final_test_mae = None
    if best_epoch != -1:
         print(f"  Best Validation MAE: {best_val_mae:.3f} at epoch {best_epoch}")
         # --- Evaluate on Test Set ---
         # print(f"  Evaluating Best Model on Test Set...") # Less verbose
         try:
             model.load_state_dict(torch.load(model_save_path, map_location=device))
             test_loss, test_mae = evaluate(model, loaders['test'], criterion, device)
             print(f"  Test MAE for Run {run_number+1}: {test_mae:.3f}")
             final_test_mae = test_mae
         except FileNotFoundError:
             print(f"  Error: Best model file not found at {model_save_path}. Skipping test evaluation.")
         except Exception as e:
             print(f"  Error during test set evaluation: {e}")
    else:
         print("  No improvement found during training. Model not saved. Skipping test evaluation.")

    # Clean up model and memory
    del model
    gc.collect()
    if device.startswith('cuda'):
        torch.cuda.empty_cache()

    return best_val_mae, final_test_mae



In [7]:

# --- Main Execution ---
if __name__ == "__main__":
    # --- Setup Datasets and DataLoaders (Once) ---
    datasets = {}
    loaders = {}
    try:
        print("\n--- Setting up Datasets and DataLoaders ---")
        for split in SPLITS:
            datasets[split] = AgePredictionDataset(npy_dir=FEATURE_ROOT / split, csv_path=CSV_PATH, split_name=split)
            shuffle = (split == 'train')
            drop_last = (split == 'train')
            loaders[split] = DataLoader(datasets[split],
                                        batch_size=BATCH_SIZE,
                                        shuffle=shuffle,
                                        num_workers=NUM_WORKERS,
                                        pin_memory=True,
                                        drop_last=drop_last,
                                        # Persistent workers can speed up if NUM_WORKERS > 0
                                        persistent_workers=True if NUM_WORKERS > 0 else False)
        print("--- Datasets and DataLoaders Ready ---")

    except (FileNotFoundError, ValueError, RuntimeError) as e:
        print(f"\nError initializing datasets/loaders: {e}")
        raise SystemExit("Dataset/Loader initialization failed.")

    # --- Define Models to Test ---
    models_to_test = {
        "MLP_2Layer": AgeMLP_2Layer,
        "MLP_3Layer": AgeMLP_3Layer,
        "MLP_3Layer_AttentionBN": AgeMLP_3Layer_AttentionBN
    }

    # --- Store results for all runs ---
    all_results = {name: {'val_mae': [], 'test_mae': []} for name in models_to_test.keys()}

    # --- Train and Evaluate Each Model Multiple Times ---
    for name, model_class in models_to_test.items():
        print(f"\n{'='*25} Testing Model: {name} ({NUM_RUNS} runs) {'='*25}")
        for run in range(NUM_RUNS):
            val_mae, test_mae = train_evaluate_single_run(model_class, name, run, loaders, DEVICE)
            if val_mae is not None:
                all_results[name]['val_mae'].append(val_mae)
            if test_mae is not None:
                all_results[name]['test_mae'].append(test_mae)
        print(f"{'='*25} Finished Testing Model: {name} {'='*25}")


    # --- Final Results Summary ---
    print("\n\n" + "="*60)
    print("      Final Average MAE Results Across Runs")
    print("="*60)
    print(f"{'Model':<25} | {'Avg Test MAE':<15} | {'Std Dev Test MAE':<18} | {'Num Valid Runs':<15}")
    print("-"*60)

    for name, results in all_results.items():
        test_maes = results['test_mae']
        num_valid_runs = len(test_maes)
        if num_valid_runs > 0:
            avg_test_mae = statistics.mean(test_maes)
            # Calculate standard deviation only if more than one run succeeded
            std_dev_test_mae = statistics.stdev(test_maes) if num_valid_runs > 1 else 0.0
            print(f"{name:<25} | {avg_test_mae:<15.3f} | {std_dev_test_mae:<18.3f} | {num_valid_runs:<15}")
        else:
            print(f"{name:<25} | {'N/A':<15} | {'N/A':<18} | {num_valid_runs:<15}")

    print("="*60)

    # Optional: Print validation MAE stats too
    print("\n\n" + "="*60)
    print("      Average Best Validation MAE Results Across Runs")
    print("="*60)
    print(f"{'Model':<25} | {'Avg Val MAE':<15} | {'Std Dev Val MAE':<18} | {'Num Valid Runs':<15}")
    print("-"*60)
    for name, results in all_results.items():
        val_maes = results['val_mae']
        num_valid_runs = len(val_maes)
        if num_valid_runs > 0:
            avg_val_mae = statistics.mean(val_maes)
            std_dev_val_mae = statistics.stdev(val_maes) if num_valid_runs > 1 else 0.0
            print(f"{name:<25} | {avg_val_mae:<15.3f} | {std_dev_val_mae:<18.3f} | {num_valid_runs:<15}")
        else:
             print(f"{name:<25} | {'N/A':<15} | {'N/A':<18} | {num_valid_runs:<15}")
    print("="*60)


    print("\nMLP comparison script finished.")



--- Setting up Datasets and DataLoaders ---
--- Datasets and DataLoaders Ready ---

========================= Testing Model: MLP_2Layer (5 runs) =========================

--- Starting Run 1/5 for Model: MLP_2Layer ---
Set random seed to 0


  Epoch 1/500 | Train Loss: 54.1984 | Val Loss: 53.3151 | Val MAE: 53.315


  Epoch 10/500 | Train Loss: 19.3070 | Val Loss: 18.0109 | Val MAE: 18.011


  Epoch 20/500 | Train Loss: 18.8666 | Val Loss: 17.4821 | Val MAE: 17.482


  Epoch 30/500 | Train Loss: 18.2698 | Val Loss: 16.9208 | Val MAE: 16.921


...

  Epoch 270/500 | Train Loss: 9.7524 | Val Loss: 7.0505 | Val MAE: 7.051


  Epoch 280/500 | Train Loss: 9.1394 | Val Loss: 7.5020 | Val MAE: 7.502


  Epoch 290/500 | Train Loss: 9.0663 | Val Loss: 8.3933 | Val MAE: 8.393


  Early stopping triggered at epoch 299.
--- Run 5 Finished for MLP_3Layer_AttentionBN (Time: 227.88s) ---
  Best Validation MAE: 6.728 at epoch 259


  Test MAE for Run 5: 7.189
========================= Finished Testing Model: MLP_3Layer_AttentionBN =========================


      Final Average MAE Results Across Runs
Model                     | Avg Test MAE    | Std Dev Test MAE   | Num Valid Runs 
------------------------------------------------------------
MLP_2Layer                | 9.080           | 0.096              | 5              
MLP_3Layer                | 8.768           | 0.032              | 5              
MLP_3Layer_AttentionBN    | 7.454           | 0.219              | 5              


      Average Best Validation MAE Results Across Runs
Model                     | Avg Val MAE     | Std Dev Val MAE    | Num Valid Runs 
------------------------------------------------------------
MLP_2Layer                | 8.351           | 0.099              | 5              
MLP_3Layer                | 8.019           | 0.053              | 5              
MLP_3Layer_AttentionBN    | 6.855           | 0.161              | 5

In [8]:
print("\n\n" + "="*60)
print("      Final Average MAE Results Across Runs")
print("="*60)
print(f"{'Model':<25} | {'Avg Test MAE':<15} | {'Std Dev Test MAE':<18} | {'Num Valid Runs':<15}")
print("-"*60)

for name, results in all_results.items():
    test_maes = results['test_mae']
    num_valid_runs = len(test_maes)
    if num_valid_runs > 0:
        avg_test_mae = statistics.mean(test_maes)
        # Calculate standard deviation only if more than one run succeeded
        std_dev_test_mae = statistics.stdev(test_maes) if num_valid_runs > 1 else 0.0
        print(f"{name:<25} | {avg_test_mae:<15.3f} | {std_dev_test_mae:<18.3f} | {num_valid_runs:<15}")
    else:
        print(f"{name:<25} | {'N/A':<15} | {'N/A':<18} | {num_valid_runs:<15}")

print("="*60)

# Optional: Print validation MAE stats too
print("\n\n" + "="*60)
print("      Average Best Validation MAE Results Across Runs")
print("="*60)
print(f"{'Model':<25} | {'Avg Val MAE':<15} | {'Std Dev Val MAE':<18} | {'Num Valid Runs':<15}")
print("-"*60)
for name, results in all_results.items():
    val_maes = results['val_mae']
    num_valid_runs = len(val_maes)
    if num_valid_runs > 0:
        avg_val_mae = statistics.mean(val_maes)
        std_dev_val_mae = statistics.stdev(val_maes) if num_valid_runs > 1 else 0.0
        print(f"{name:<25} | {avg_val_mae:<15.3f} | {std_dev_val_mae:<18.3f} | {num_valid_runs:<15}")
    else:
        print(f"{name:<25} | {'N/A':<15} | {'N/A':<18} | {num_valid_runs:<15}")
print("="*60)




      Final Average MAE Results Across Runs
Model                     | Avg Test MAE    | Std Dev Test MAE   | Num Valid Runs 
------------------------------------------------------------
MLP_2Layer                | 9.080           | 0.096              | 5              
MLP_3Layer                | 8.768           | 0.032              | 5              
MLP_3Layer_AttentionBN    | 7.454           | 0.219              | 5              


      Average Best Validation MAE Results Across Runs
Model                     | Avg Val MAE     | Std Dev Val MAE    | Num Valid Runs 
------------------------------------------------------------
MLP_2Layer                | 8.351           | 0.099              | 5              
MLP_3Layer                | 8.019           | 0.053              | 5              
MLP_3Layer_AttentionBN    | 6.855           | 0.161              | 5              


In [9]:
# --- Detailed Run Results ---
print("\n\n" + "="*60)
print("      Detailed MAE Results Per Run")
print("="*60)

for name, results in all_results.items():
    print(f"\nModel: {name}")
    test_maes = results.get('test_mae', [])
    val_maes = results.get('val_mae', [])

    if test_maes:
        # Format MAEs to 3 decimal places for printing
        formatted_test_maes = [f"{mae:.3f}" for mae in test_maes]
        print(f"  Test MAEs (Runs 1-{len(test_maes)}): {', '.join(formatted_test_maes)}")
    else:
        print("  Test MAEs: No successful test runs.")

    if val_maes:
        # Format MAEs to 3 decimal places for printing
        formatted_val_maes = [f"{mae:.3f}" for mae in val_maes]
        print(f"  Best Validation MAEs (Runs 1-{len(val_maes)}): {', '.join(formatted_val_maes)}")
    else:
        print("  Best Validation MAEs: No successful validation runs.")

# --- Final Average Results Summary (copied from previous cell for completeness) ---
print("\n\n" + "="*60)
print("      Final Average MAE Results Across Runs")
print("="*60)
print(f"{'Model':<25} | {'Avg Test MAE':<15} | {'Std Dev Test MAE':<18} | {'Num Valid Runs':<15}")
print("-"*60)

for name, results in all_results.items():
    test_maes = results.get('test_mae', [])
    num_valid_runs = len(test_maes)
    if num_valid_runs > 0:
        avg_test_mae = statistics.mean(test_maes)
        std_dev_test_mae = statistics.stdev(test_maes) if num_valid_runs > 1 else 0.0
        print(f"{name:<25} | {avg_test_mae:<15.3f} | {std_dev_test_mae:<18.3f} | {num_valid_runs:<15}")
    else:
        print(f"{name:<25} | {'N/A':<15} | {'N/A':<18} | {num_valid_runs:<15}")

print("="*60)

print("\n\n" + "="*60)
print("      Average Best Validation MAE Results Across Runs")
print("="*60)
print(f"{'Model':<25} | {'Avg Val MAE':<15} | {'Std Dev Val MAE':<18} | {'Num Valid Runs':<15}")
print("-"*60)
for name, results in all_results.items():
    val_maes = results.get('val_mae', [])
    num_valid_runs = len(val_maes)
    if num_valid_runs > 0:
        avg_val_mae = statistics.mean(val_maes)
        std_dev_val_mae = statistics.stdev(val_maes) if num_valid_runs > 1 else 0.0
        print(f"{name:<25} | {avg_val_mae:<15.3f} | {std_dev_val_mae:<18.3f} | {num_valid_runs:<15}")
    else:
        print(f"{name:<25} | {'N/A':<15} | {'N/A':<18} | {num_valid_runs:<15}")
print("="*60)



      Detailed MAE Results Per Run

Model: MLP_2Layer
  Test MAEs (Runs 1-5): 9.099, 8.949, 9.018, 9.171, 9.165
  Best Validation MAEs (Runs 1-5): 8.363, 8.211, 8.303, 8.467, 8.409

Model: MLP_3Layer
  Test MAEs (Runs 1-5): 8.738, 8.810, 8.735, 8.772, 8.787
  Best Validation MAEs (Runs 1-5): 8.000, 8.112, 8.007, 7.981, 7.995

Model: MLP_3Layer_AttentionBN
  Test MAEs (Runs 1-5): 7.572, 7.610, 7.246, 7.654, 7.189
  Best Validation MAEs (Runs 1-5): 7.099, 6.904, 6.850, 6.695, 6.728


      Final Average MAE Results Across Runs
Model                     | Avg Test MAE    | Std Dev Test MAE   | Num Valid Runs 
------------------------------------------------------------
MLP_2Layer                | 9.080           | 0.096              | 5              
MLP_3Layer                | 8.768           | 0.032              | 5              
MLP_3Layer_AttentionBN    | 7.454           | 0.219              | 5              


      Average Best Validation MAE Results Across Runs
Model             